### Introduction
In this lab, you will implement [the Pohlig-Hellman algorithm](https://www-ee.stanford.edu/~hellman/publications/28.pdf) which efficiently computes discrete logarithms in a [finite abelian group](https://en.wikipedia.org/wiki/Abelian_group#Finite_abelian_groups) whose order is a [smooth integer](https://en.wikipedia.org/wiki/Smooth_number). The Pohlig-Hellman algorithm is the reason why the prime $p$ and the generator $g$ in the Diffie-Hellman key exchange need to be picked carefully (so that the order of the group $G$, which equals to $p-1$, has a large prime factor $q$, and the generator has order $q$).

In a bit more detail, the Pohlig-Hellman algorithm works as follows. Suppose one is trying to solve the discrete logarithm $y = g^x \ (\text{mod} \ p)$ where $p-1 = p_{0} \times p_{1} \times \ldots \times p_{k-1}$ ($p_i$'s are prime numbers. For simplicity, assume that they are all distinct for now.).
The Pohlig-Hellman algorithm is a divide and conquer problem that finds $x \ (\text{mod} \ p_1), x \ (\text{mod} \ p_2), \ldots, x \ (\text{mod} \ p_{k-1})$. And from there, $x (\text{mod} \ p_{0} \times p_{1} \times \ldots \times p_{k-1}) = x \ (\text{mod} \ p-1)$. We explain the core idea in finding $x \ (\text{mod} \ p_i)$ below.

### Finding $x \ (\text{mod} \ p_i)$
Consider the following concrete example where $p = 2 \times 5 \times 7 + 1$ and $g = 11$. $g$ has order $p-1$ (Check that yourself!).
The key observation is that for some special powers of $g$, say $g^t$, the order of $g^t$ is much smaller, and the discrete logarithm problem is much simpler with $g^t$ as the generator. There is nothing mysterious about $t$. You can verify yourself that if $t = 2 \times 5$, $g^t$ has order $7$; $t = 5 \times 7$, $g^t$ has order $2$; and $t = 2 \times 7$, $g^t$ has order $5$!

How does the fact help us? Suppose the discrete logarithm we are trying to solve is $g^x = g^{13} = 63 \ (\text{mod} \ p)$. What we can do now is to raise the equation to the power $t=2 \times 5$. So ${g^x}^t = 63^t = 20 \ (\text{mod} \ p)$. But ${g^x}^t = g^{xt} = {g^t}^x$. So solving the discrete logarithm with $g^t$ as the generator gives us $x \ (\text{mod} 7)$. We can do the same with $t = 5 \times 7$ and $t = 2 \times 7$ to find $x \ (\text{mod} \ 2)$ and $x \ (\text{mod} \ 5)$ respectively.

All that is left to explain is how to solve the easier instances of the discrete logarithm problem. You can do that with the baby-step giant-step algorithm you have seen in the class!

### Finding $x \ (\text{mod} \ p_{0} \times p_{1} \times \ldots \times p_{k-1})$
Once we know $x \ (\text{mod} \ p_i)$ for each $i$, we can compute $x \ (\text{mod} \ p_{0} \times p_{1} \times \ldots \times p_{k-1})$ by using the [Chinese remainder theorem](https://en.wikipedia.org/wiki/Chinese_remainder_theorem). That effectively solves the original discrete problem $g^x \ (\text{mod} \ p)$.


### Remark
So how do you know the factorization of $p-1$ in the first place? Didn't we say that factorization of a large (2048-bit) integer is hard?
Yes, factorization is hard in general. But since we only require $p$ to be a prime, there is no guarantee that $p-1$ does not have small factors. The Pohlig-Hellman algorithm *assumes* that $p-1$ can be factorized into small factors and small factors only (that is what a smooth integer means). In other words, the attack does not work against all instances of Diffie-Hellman key exchange (in the version where the generator $g$ is picked randomly), but when it works, it is devastating.

### Your Task
In this programming assignment, you will implement the Pohlig-Hellman algorithm in its full generality. You should fully understand the reasons behind the choice of prime $q$ and the generator $g$ after the programming assignment.

### Overview
The assignment is broken down into four components. In the first component, you will implement exponentiation by squaring as a warm-up. Then, you will implement the baby-step giant-step algorithm as you have seen in the class. After that, you will implement a simplified version of the Pohlig-Hellman algorithm where $p-1 = p_1^{e_1}$ ($p_1$ is a prime, $e_1$ is an arbitrary positive integral exponent). Finally, you will extend your implementation to the general case where $p-1 = \prod_i p_i^{e_i}$.


## Part 1: Exponentiation by Squaring (10 points)
Implement a function that computes $g^x \ (\text{mod} \ p)$ using [exponentiation by squaring](https://en.wikipedia.org/wiki/Modular_exponentiation).

In [69]:
def modexp(g, x, p):
    ans = 1
    count = 0
    while count < x:
        count += 1
        ans = (g * ans) % p
    return ans

p = 10
g = 3
x = 3
print(modexp(g, x, p))

7


## Part 2: Baby-Step Giant-Step (30 points)
Implement the baby-step giant-step algorithm. Your function should take the prime $p$, the generator $g$, the order of the generator $r$ and a group element $g^x$ as inputs and outputs $x$. You will be calling the function with generators of the shape $g^t$ and group elements of the shape ${g^x}^t$ later.

In [70]:
def baby_step_giant_step(p, g, r, gx):
    m = int(r**0.5) + 1
    baby_steps = {}
    value = 1

    for j in range(m):
        baby_steps[value] = j
        value = (value * g) % p

    g_to_m    = modexp(g, m, p)
    inverse = modexp(g_to_m, p-2, p)

    gma = gx
    for i in range(m):
        if gma in baby_steps:
            return i * m + baby_steps[gma]
        gma = (gma * inverse) % p

p  = 1019
g  = 2
r  = 1018
gx = 623
x = baby_step_giant_step(p, g, r, gx)
print(f"Found x = {x}")

Found x = 129


## Part 3: The Pohlig-Hellman Algorithm - the Simple Case (30 points)
Implement the Pohlig-Hellman algorithm for the case $p-1 = p_1^{e_1}$.

In [71]:
def simple_case_pohlig_hellman(p, g, gx, p_1, e_1):
    r = p_1**e_1
    g_0 = modexp(g, r // p_1, p)
    x = 0

    for i in range(e_1):
        exp = r // (p_1**(i + 1))
        inverse = modexp(g, r - (x % r), p)
        h_i = modexp((gx * inverse) % p, exp, p)
        x_i = baby_step_giant_step(p, g_0, p_1, h_i)
        x += x_i * (p_1**i)

    return x

p, p_1, e_1, g, true_x = 17, 2, 4, 3, 13
gx = modexp(g, true_x, p)
recovered = simple_case_pohlig_hellman(p, g, gx, p_1, e_1)
print("recovered x =", recovered)

recovered x = 13


## Part 4: The Pohlig-Hellman Algorithm - Full Implementation (30 points)
Implement the Pohlig-Hellman algorithm for the general case where $p-1 = \prod_i p_i^{e_i}$.

In [78]:
def recombine(R, M):
    prod = 1

    for m in M:
        prod *= m

    x = 0

    for r_i, m_i in zip(R, M):
        N_i = prod // m_i
        inv_N_i = pow(N_i, -1, m_i)
        x = (x + r_i * N_i * inv_N_i) % prod

    return x

def pohlig_hellman(p, g, gx, factors):
    R = []
    M    = []

    for p_i, e_i in factors:
        r_i   = p_i**e_i
        g_i   = modexp(g, (p-1)//r_i, p)
        gx_i  = modexp(gx, (p-1)//r_i, p)
        x_i = simple_case_pohlig_hellman(p, g_i, gx_i, p_i, e_i)
        R.append(x_i)
        M.append(r_i)

    return recombine(R, M)

p = 1019
factors = [(2, 1), (509, 1)]
g = 2
true_x = 12345
gx = modexp(g, true_x, p)
x_recov = pohlig_hellman(p, g, gx, factors)
print(f"Recovered x ≡ {x_recov} (mod {p-1})")

Recovered x ≡ 129 (mod 1018)


### Remark
You can check the correctness of your implementation with small prime factors (make sure your $p$ is a prime!). The correctness of your implementation will be checked against larger prime factors (48-bit or more). You can generate primes by implementing the Miller-Rabin primality test yourself (recommended for your understanding) or use the `getPrime()` function from the `PyCryptodome` library (https://www.pycryptodome.org/src/util/util#Crypto.Util.number.getPrime).